In [1]:
import os
corpora_path = '../data/Topicos'
corpora_dirs = os.listdir(corpora_path)
corpora_dirs = [d for d in corpora_dirs]

In [2]:
noticias_xls = []
for supervisor in corpora_dirs:
    noticias = os.listdir(corpora_path + '/' + supervisor)
    for noticia in noticias:
        noticias_xls.append(corpora_path + '/' + supervisor + '/' + noticia)

In [18]:
noticias_xls[:5]

['../data/Topicos/Erick/C10.xlsx',
 '../data/Topicos/Erick/C17.xlsx',
 '../data/Topicos/Erick/C18.xlsx',
 '../data/Topicos/Erick/C1_.xlsx',
 '../data/Topicos/Erick/C42.xlsx']

In [4]:
def extrair_nome(path):
    nome = path.replace('../data/Topicos/','')
    nome = nome.replace('.xlsx', '')
    return nome

In [5]:
import pandas as pd

#dicionario para armazenar o path de cada noticia
noticias = {}
#dicionario para armazenar um DF de cada noticia
notic_indiv = {}
#noticias repetidas
notic_rep = []
for noticia in noticias_xls:
    #cria uma key como a chave de cada noticia
    noticias[noticia] = []
    #le o arquivo do excel
    xlsx = pd.ExcelFile(noticia)
    #para cada key é atribuido um DF referente a planilha com a noticia
    #sheet_name = None, ler todas as folhas de uma planilha
    noticias[noticia] = pd.read_excel(xlsx, sheet_name=None)
    for notic in noticias[noticia].keys():
        #noticias repetidas apenas serão inseridas novas colunas
        if notic in notic_indiv:
            if notic not in notic_rep: notic_rep.append(notic)
            #aqui se cria um DataFrame
            plan_aux = pd.read_excel(xlsx, sheet_name=notic)
            superv = extrair_nome(noticia)
            #DataFrameName.insert(loc, column, value, allow_duplicates = False)
            notic_indiv[notic].insert(len(plan_aux.keys()), 'Tópico (Rotule apenas Inicio)', plan_aux['Tópico (Rotule apenas Inicio)'], True)
            notic_indiv[notic].insert(len(plan_aux.keys()), 'Segmentação', plan_aux['Segmentação'], True)
        else:
            #aqui se cria uma Serie
            notic_indiv[notic] = pd.read_excel(xlsx, sheet_name=notic)
        #normalizar a palavra 'inicio' para ficarem todas sem acentos
        notic_indiv[notic] = notic_indiv[notic].replace('Início','Inicio')

In [6]:
noticias_valores = {}
for noticia in notic_rep:
    noticias_valores[noticia] = {'segmentacao': [], 'matchs': []}
    #ordem é a ordem da tabela dada em (m,n)
    ordem = notic_indiv[noticia]['Segmentação'].shape
    #numero de linhas do DF
    for row in notic_indiv[noticia]['Segmentação'].iterrows():
        shape = row.value_counts()
        noticias_valores[noticia]['segmentacao'].append(list(shape.index))
        #numero de colunas do DF, que tambem representa o numero de etiquetadores
        percent = [(int(x)/int(ordem[1]))*100 for x in shape.values]
        #print(percent)
        noticias_valores[noticia]['matchs'].append(percent)
    #salva em cada chave noticia um df com suas metricas
    noticias_valores[noticia] = pd.DataFrame(noticias_valores[noticia])

In [7]:
notic_indiv['C17.1']['Segmentação']

,Segmentação,Segmentação,Segmentação,Segmentação,Segmentação,Segmentação,Segmentação
0,NaN,Inicio,Inicio,Inicio,Inicio,NaN,Inicio
1,NaN,Meio,Meio,Meio,Meio,NaN,Fim
2,NaN,Fim,Meio,Fim,Meio,NaN,Inicio
3,NaN,Inicio,Fim,Inicio,Fim,NaN,Meio
4,NaN,Fim,Inicio,Meio,Inicio,NaN,Fim
5,NaN,Inicio,Inicio,Fim,Meio,NaN,Inicio
6,NaN,Inicio,Fim,Inicio,Meio,NaN,Fim
7,NaN,Inicio,Inicio,Meio,Meio,NaN,Inicio
8,NaN,Inicio,Fim,Fim,Fim,NaN,Fim


In [8]:
cont = 0;
matchs = []
for notic in notic_rep:
    for row in notic_indiv[notic]['Segmentação'].iterrows():
        lista = []
        lista = row[1].tolist()
        print(lista)
        for x in range(1, 7):
            for y in range(x+1,7):
                #print(lista[x], lista[y])
                if lista[x] != 'nan' and lista[x] == lista[y]:
                    cont+=1
    size = notic_indiv[notic]['Segmentação'].size
    matchs.append((notic, (cont/size)*100))
    cont = 0
    
                
print(cont)

[nan, 'Inicio', 'Inicio', 'Inicio', 'Inicio', nan, 'Inicio']
[nan, 'Meio', 'Meio', 'Meio', 'Meio', nan, 'Fim']
[nan, 'Fim', 'Meio', 'Fim', 'Meio', nan, 'Inicio']
[nan, 'Inicio', 'Fim', 'Inicio', 'Fim', nan, 'Meio']
[nan, 'Fim', 'Inicio', 'Meio', 'Inicio', nan, 'Fim']
[nan, 'Inicio', 'Inicio', 'Fim', 'Meio', nan, 'Inicio']
[nan, 'Inicio', 'Fim', 'Inicio', 'Meio', nan, 'Fim']
[nan, 'Inicio', 'Inicio', 'Meio', 'Meio', nan, 'Inicio']
[nan, 'Inicio', 'Fim', 'Fim', 'Fim', nan, 'Fim']
[nan, 'Inicio', nan, 'Inicio', nan, nan, 'Inicio']
[nan, 'Fim', nan, 'Meio', nan, nan, 'Inicio']
[nan, 'Inicio', nan, 'Fim', nan, nan, 'Meio']
[nan, 'Meio', nan, 'Inicio', nan, nan, 'Meio']
[nan, 'Fim', nan, 'Fim', nan, nan, 'Fim']
[nan, 'Inicio', nan, 'Inicio', nan, nan, 'Inicio']
[nan, 'Inicio', nan, 'Fim', nan, nan, 'Inicio']
[nan, 'Inicio', 'Inicio', 'Inicio', 'Inicio', nan, 'Inicio']
[nan, nan, 'Fim', 'Meio', 'Meio', nan, 'Fim']
[nan, nan, 'Inicio', 'Meio', 'Meio', nan, 'Inicio']
[nan, nan, 'Meio', 'Meio', 

In [9]:
matchs

[('C17.1', 58.730158730158735),
 ('C17.2', 22.448979591836736),
 ('C42.1', 51.42857142857142),
 ('C42.2', 9.523809523809524)]

In [19]:
noticias_valores

{'C17.1':            segmentacao                                             matchs
 0             [Inicio]                                [71.42857142857143]
 1          [Meio, Fim]            [57.14285714285714, 14.285714285714285]
 2  [Meio, Fim, Inicio]  [28.57142857142857, 28.57142857142857, 14.2857...
 3  [Inicio, Fim, Meio]  [28.57142857142857, 28.57142857142857, 14.2857...
 4  [Inicio, Fim, Meio]  [28.57142857142857, 28.57142857142857, 14.2857...
 5  [Inicio, Meio, Fim]  [42.857142857142854, 14.285714285714285, 14.28...
 6  [Inicio, Fim, Meio]  [28.57142857142857, 28.57142857142857, 14.2857...
 7       [Inicio, Meio]            [42.857142857142854, 28.57142857142857]
 8        [Fim, Inicio]            [57.14285714285714, 14.285714285714285],
 'C17.2':            segmentacao                                             matchs
 0             [Inicio]                               [42.857142857142854]
 1  [Inicio, Meio, Fim]  [14.285714285714285, 14.285714285714285, 14.28...
 2  [I

In [11]:
notic_indiv['C17.1']['Tópico (Rotule apenas Inicio)']

,Tópico (Rotule apenas Inicio),Tópico (Rotule apenas Inicio),Tópico (Rotule apenas Inicio),Tópico (Rotule apenas Inicio),Tópico (Rotule apenas Inicio),Tópico (Rotule apenas Inicio),Tópico (Rotule apenas Inicio)
0,NaN,"Subida de tom, debate eleitoral","caciques pefelistas, escândalos, petista.","Debate Eleitoral, Alckmin, irritação","Discursos duros, Irritação",NaN,"Geraldo Alckmin, Lula, mensalão, Waldomiro Diniz."
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,"Campanha, Alckmin, Antonio Carlos Magalhães."
3,NaN,Alckmin criticou Lula,NaN,"Alckmin, discurso, segundo turno",NaN,NaN,NaN
4,NaN,NaN,Discurso.,NaN,"Discurso, Escola de Samba",NaN,NaN
5,NaN,Enfrentamento num segundo turno,"Pesquisas, eleição.",NaN,NaN,NaN,"Pesquisas, segundo turno, eleição, programa el..."
6,NaN,Reviravolta no discurso não deve chegar a TV.,NaN,"Discurso, sambistas, dança",NaN,NaN,NaN
7,NaN,Alckmin ressaltou suas realizações.,"Realizações, Estado.",NaN,NaN,NaN,"Escola de Samba, Geraldo Alckmin, discurso."
8,NaN,Candidato dançou.,NaN,NaN,NaN,NaN,NaN


In [20]:
def extract_theme_tag(s):
    return s.split('_')[:2]

In [26]:
import os

corpora_path = '../data/corpora/CSTNews 6.0'
corpora_dirs = os.listdir(corpora_path)
corpora_dirs.remove('For all the clusters')
corpora_dirs = [extract_theme_tag(d) for d in corpora_dirs if d not in '.DS_Store']

In [27]:
corpora_dirs

[['C22', 'Cotidiano'],
 ['C36', 'Cotidiano'],
 ['C10', 'Mundo'],
 ['C11', 'Cotidiano'],
 ['C12', 'Mundo'],
 ['C13', 'Mundo'],
 ['C14', 'Mundo'],
 ['C15', 'Mundo'],
 ['C16', 'Politica'],
 ['C17', 'Politica'],
 ['C18', 'Mundo'],
 ['C19', 'Esportes'],
 ['C1', 'Mundo'],
 ['C20', 'Politica'],
 ['C21', 'Cotidiano'],
 ['C23', 'Mundo'],
 ['C24', 'Esportes'],
 ['C25', 'Esportes'],
 ['C26', 'Mundo'],
 ['C27', 'Esportes'],
 ['C28', 'Esportes'],
 ['C29', 'Mundo'],
 ['C2', 'Politica'],
 ['C30', 'Dinheiro'],
 ['C31', 'Esportes'],
 ['C32', 'Mundo'],
 ['C33', 'Cotidiano'],
 ['C34', 'Cotidiano'],
 ['C35', 'Mundo'],
 ['C37', 'Cotidiano'],
 ['C38', 'Esportes'],
 ['C39', 'Cotidiano'],
 ['C3', 'Cotidiano'],
 ['C40', 'Politica'],
 ['C41', 'Esportes'],
 ['C42', 'Politica'],
 ['C43', 'Politica'],
 ['C44', 'Politica'],
 ['C45', 'Cotidiano'],
 ['C46', 'Mundo'],
 ['C47', 'Mundo'],
 ['C48', 'Esportes'],
 ['C49', 'Cotidiano'],
 ['C4', 'Cotidiano'],
 ['C50', 'Politica'],
 ['C5', 'Cotidiano'],
 ['C6', 'Cotidiano'],


In [31]:
from nltk.corpus import stopwords
stop_words = stopwords.words('portuguese')

In [33]:
len(stop_words)

204